# Weather Time Series

## Imports and Load CSV Data

In [2]:
%load_ext autoreload
%autoreload 2
import os
import pandas as pd
import matplotlib.pyplot as plt
from statsmodels.tsa.seasonal import seasonal_decompose
import numpy as np
from datetime import date, timedelta
from sklearn.impute import SimpleImputer
import scipy.optimize as op
from statsmodels.tsa.stattools import adfuller
from statsmodels.graphics.tsaplots import plot_acf
from statsmodels.graphics.tsaplots import plot_pacf
from pmdarima.arima.utils import ndiffs
from statsmodels.tsa.arima_model import ARIMA
import pmdarima as pm
from numpy import fft

In [3]:
csv_path = os.path.join('..','data','data_weather_cleaned.csv')
df = pd.read_csv(csv_path)
df

,date,numer_sta,Latitude,Longitude,Altitude,pmer,dd,ff,t,u,ssfrai,rr3,pres,dd_sin,dd_cos
0,2010-01-01 00:00:00,7510.0,44.830667,-0.691333,47.0,99050.0,230.0,9.8,9.6,81.0,0.0,0,98410.0,-0.766044,-6.427876e-01
1,2010-01-01 03:00:00,7510.0,44.830667,-0.691333,47.0,99160.0,250.0,11.8,8.7,87.0,0.0,0,98520.0,-0.939693,-3.420201e-01
2,2010-01-01 06:00:00,7510.0,44.830667,-0.691333,47.0,99570.0,290.0,5.1,7.6,91.0,0.0,0,98920.0,-0.939693,3.420201e-01
3,2010-01-01 09:00:00,7510.0,44.830667,-0.691333,47.0,99990.0,310.0,5.7,6.8,92.0,0.0,0,99340.0,-0.766044,6.427876e-01
4,2010-01-01 12:00:00,7510.0,44.830667,-0.691333,47.0,100350.0,310.0,6.2,6.6,82.0,0.0,0,99690.0,-0.766044,6.427876e-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
505852,2021-07-31 09:00:00,7650.0,43.437667,5.216000,9.0,100920.0,310.0,3.2,27.2,58.0,0.0,0,100560.0,-0.766044,6.427876e-01
505853,2021-07-31 12:00:00,7650.0,43.437667,5.216000,9.0,100770.0,270.0,6.1,30.6,49.0,0.0,0,100410.0,-1.000000,-1.836970e-16
505854,2021-07-31 15:00:00,7650.0,43.437667,5.216000,9.0,100680.0,310.0,8.2,30.0,40.0,0.0,0,100320.0,-0.766044,6.427876e-01
505855,2021-07-31 18:00:00,7650.0,43.437667,5.216000,9.0,100750.0,320.0,8.0,27.9,45.0,0.0,0,100390.0,-0.642788,7.660444e-01


## Cleaning

<b>Consigne:</b> Sélectionner les colonnes "date" et "temperature" de la station 7577 (output: 33775 rows)

In [10]:
data = df[["date","t","numer_sta"]]
data = data[data["numer_sta"] == 7577]
data.drop(columns="numer_sta")

,date,t
370914,2010-01-01 00:00:00,7.3
370915,2010-01-01 03:00:00,7.7
370916,2010-01-01 06:00:00,7.5
370917,2010-01-01 09:00:00,7.6
370918,2010-01-01 12:00:00,8.3
...,...,...
404684,2021-07-31 09:00:00,20.8
404685,2021-07-31 12:00:00,23.4
404686,2021-07-31 15:00:00,24.2
404687,2021-07-31 18:00:00,20.4


<b>Consigne:</b> Enlever les duplicatas (output: 33769 rows)

In [11]:
data = data.drop_duplicates()
data.shape

(33769, 3)

<b>Consigne:</b> Convertir les date au bon format (utiliser: pd.to_datetime) 

In [16]:
data["date"] = pd.to_datetime(data["date"])

<b>Consigne:</b> Retrouver les dates manquantes de la série (33840 rows)
- Créer une liste de dates complêtes commencant à la date du début de la série et finissant à la date de la série avec le même échantillonage de 3h (utiliser pd.date_range)
- Faire une jointure "outer" de la série et de la nouvelle colonne créée (utiliser pd.merge)
- Ordonner les dates du nouveaux dataframe créés (utiliser sort_values)

In [22]:
starto = data["date"]
pd.date_range(start='1/1/2018', periods=5, freq='M')
str(starto.iloc[0])

'2010-01-01 06:00:00'

In [ ]:
df.isnull().sum().sort_values(ascending=False)

<b>Consigne:</b> Combler les valeurs aux dates manquantes avec la valeur moyenne de la série
- Reset l'index
- Remplacer les valeurs nulles par la moyenne (utiliser SimpleImputer)

In [ ]:
df.isnull().sum().sort_values(ascending=False)

<b>Visualisation de la ST de température en °C clean</b>

<img src="../images/st_plot_1.png">

## Retirer la saisonnalité annuelle

<b> Méthode de régression sinusoidale</b>

In [ ]:
def sinus_fit(index_list, data, index_list_fit, guess_freq):
    guess_mean = np.mean(data)
    guess_std = 3*np.std(data)/(2**0.5)/(2**0.5)
    guess_phase = 0
    guess_amp = 1

    data_first_guess = guess_std*np.sin(guess_freq*2*np.pi*index_list/len(t)+guess_phase) + guess_mean

    optimize_func = lambda x: x[0]*np.sin(x[1]*2*np.pi*index_list/len(t)+x[2]) + x[3] - data
    est_amp, est_freq, est_phase, est_mean = op.leastsq(optimize_func, [guess_amp, guess_freq, guess_phase, guess_mean])[0]
    data_fit = est_amp*np.sin(est_freq*2*np.pi*index_list_fit/len(t)+est_phase)+est_mean
    return data_fit, data_first_guess

<b>Consigne:</b> Utiliser la méthode sinus_fit pour déterminer la saisonalité annuelle de la série
- La guess_freq correspond au nombre de sinusoides présentes visibles sur le signal

<b>Tracé de la saisonalité annuelle de la température (en vert)</b>

<img src="../images/st_plot_2.png">

<b>Consigne:</b> Retirer cette saisonalité de la série temporelle pour travailler sur les résidus

<b>Tracé des résidus de température</b>

<img src="../images/st_plot_3.png">

## Préparation des données pour le prédiction des deux prochains jours

<b>Consigne:</b> Définir les dates en index et la fréquence de 3H du dataframe (utiliser asfreq)

In [ ]:
df_f.info()

<b>Consigne:</b> Ne garder que les 25 derniers jours (suffisants pour prédire les 2 jours suivants)
- Utiliser les 20 premiers jours de ces 25 jours comme données de training
- Utiliser les 5 derniers jours comme données de testing

<b>Consigne:</b> Retirer la saisonnalité journalière (utiliser seasonal_decompose avec le modèle "additive")

<b>Tracé des résidus de température</b>

<img src="../images/st_plot_4.png">

<b>Consigne:</b> Isolé la série temporelle de la saisonnalité journalière (utiliser .seasonal)

<b>Consigne:</b> Soustraire cette saisonnalité à la série

<b>Tracé des résidus après extraction de la saisonalité journalière</b>

<img src="../images/st_plot_5.png">

## Model ARIMA

<b>Consigne:</b> Utiliser les methodes plot_acf et plot_pacf pour visualiser la faisabilité du modèle

<b>Tracés plot_acf et plot_pacf</b>

<img src="../images/st_plot_6.png">

<b>Consigne:</b> Evaluer les meilleurs paramètres pour le modèle ARIMA (utiliser pm.auto_arima)

<b>Consigne:</b> Faire tourner le modèle ARIMA avec le training set (utiliser ARIMA et arima.fit)

In [ ]:
arima.summary()

<img src="../images/st_tab_1.png" width="500px">

<b>Consigne:</b> Utiliser le resulat du modèle pour avec le testing set (utiliser arima.forecast)

<b>Tracé de la prédiction du modèle ARIMA</b>

<img src="../images/st_plot_7.png">

## Reconstruire la prédiction avec la saisonalité

<b>Méthode utilsant les transformées de Fourier</b>
<br>
L'objectif est de reconstruire la saisonalité journalière a fin de prédire cette saisonalité pour les jours suivants

In [ ]:
#https://gist.github.com/tartakynov/83f3cd8f44208a1856ce
def fourierExtrapolation(x, n_predict):
    n = x.size
    n_harm = n
    t = np.arange(0, n)
    x_freqdom = fft.fft(x)
    f = fft.fftfreq(n)
    indexes = list(range(n))
    indexes.sort(key = lambda i: np.absolute(f[i]))
    t = np.arange(0, n + n_predict)
    restored_sig = np.zeros(t.size)
    for i in indexes[:1 + n_harm * 2]:
        ampli = np.absolute(x_freqdom[i]) / n   # amplitude
        phase = np.angle(x_freqdom[i])          # phase
        restored_sig += ampli * np.cos(2 * np.pi * f[i] * t + phase)
    return restored_sig

<b>Consigne:</b> Utiliser la méthode ci dessus pour reconstruire la saisonalité journalière pour les jours de testing

<b>Tracé de la saisonalité journalière reconstruite avec la méthode des transformées de fourier</b>

<img src="../images/st_plot_8.png">

<b>Consigne:</b> Additionner les prédictions du modèle arima, la saisonalité annuelle et la saisonalité journalière pour retrouver le prédiction totale

<b>Tracé de la prédiction finale</b>

<img src="../images/st_plot_9.png">